In [1]:
import optuna
import joblib
import numpy as np
import pandas as pd
import sys
from concurrent.futures import ThreadPoolExecutor
from optuna.storages import JournalStorage, JournalFileStorage
from tdc.single_pred import ADME
from project_resources.import_utils import NotebookFinder
from sklearn.preprocessing import MinMaxScaler
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import fp_from_smiles, parse_jazzy_df, HyperparamTuner

C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [2]:
feature_types = ["morgan", "jazzy"]
tdc_benchmarks = ["obach", "microsome", "hepatocyte"]
model_identifiers = ["linear", "KRR", "GB", "RF", "ANN"]
tdc_datasets = {}
smiles = {}
halflives = {}
mol_features = {}

# sampler - a method used to generate new sets of hyperparameters in each iteration of the optimization process
samplers = {
    'RandomSampler': optuna.samplers.RandomSampler,          # Sampler that selects hyperparameters randomly from the search space.
    'GridSampler': optuna.samplers.GridSampler,              # Sampler that performs a grid search over the hyperparameter space.
    'TPESampler': optuna.samplers.TPESampler,                # Sampler that uses a tree-structured Parzen estimator to model the objective function and sample new points from the search space.
    'CmaEsSampler': optuna.samplers.CmaEsSampler,            # Sampler that uses the Covariance Matrix Adaptation Evolution Strategy algorithm to efficiently search the hyperparameter space.
    'NSGAIISampler': optuna.samplers.NSGAIISampler,          # Multi-objective evolutionary algorithm that generates new samples using non-dominated sorting and crowding distance selection.
    'QMCSampler': optuna.samplers.QMCSampler,                # Quasi-Monte Carlo sampler that uses low-discrepancy sequences to sample the search space in a more efficient and evenly distributed way than random sampling.
    'BoTorchSampler': optuna.integration.BoTorchSampler,     # Sampler that leverages the BoTorch library for Bayesian optimization and can handle both continuous and categorical hyperparameters.
    'BruteForceSampler': optuna.samplers.BruteForceSampler,  # Sampler that exhaustively evaluates all possible combinations of hyperparameters in the search space.
}
# pruner - a technique used to eliminate unpromising trials during the course of hyperparameter optimization.
pruners = {
    'BasePruner': optuna.pruners.BasePruner,                            # This is the base class for all pruning strategies in Optuna. It provides a skeleton for implementing custom pruning strategies.
    'MedianPruner': optuna.pruners.MedianPruner,                        # A pruner that prunes unpromising trials that have median objective values, as determined in previous steps.
    'SuccessiveHalvingPruner': optuna.pruners.SuccessiveHalvingPruner,  # This pruner repeatedly splits trials into halves, discarding the lower performing half at each iteration.
    'HyperbandPruner': optuna.pruners.HyperbandPruner,                  # This pruner implements the Hyperband algorithm, which selects promising trials and runs them with different resource allocation schemes to determine the best one.
    'PercentilePruner': optuna.pruners.PercentilePruner,                # A pruner that prunes unpromising trials based on their percentile rank relative to all completed trials.
    'NopPruner': optuna.pruners.NopPruner,                              # A pruner that does nothing and does not prune any trials.
    'ThresholdPruner': optuna.pruners.ThresholdPruner,                  # This pruner prunes trials that have not reached a certain level of performance (i.e., objective value).
    'PatientPruner': optuna.pruners.PatientPruner,                      # This pruner prunes trials that do not show improvement over a certain number of steps (or epochs).
}

In [3]:
obach = ADME(name='Half_Life_Obach')
obach_split = obach.get_split()
tdc_datasets["obach"] = obach_split
microsome = ADME(name='Clearance_Microsome_AZ')
microsome_split = microsome.get_split()
tdc_datasets["microsome"] = microsome_split
hepatocyte = ADME(name='Clearance_Hepatocyte_AZ')
hepatocyte_split = hepatocyte.get_split()
tdc_datasets["hepatocyte"] = hepatocyte_split

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


In [4]:
halflives["morgan"] = {}
for benchmark in tdc_benchmarks:
    print(benchmark)
    smiles[benchmark] = {}
    halflives["morgan"][benchmark] = {}
    
    benchmark_train_smiles = tdc_datasets[benchmark]["train"]["Drug"]
    benchmark_test_smiles = tdc_datasets[benchmark]["test"]["Drug"]
    smiles[benchmark]["train"] = np.array(benchmark_train_smiles)
    smiles[benchmark]["test"] = np.array(benchmark_test_smiles)
    
    benchmark_train_halflives = tdc_datasets[benchmark]["train"]["Y"]
    benchmark_test_halflives = tdc_datasets[benchmark]["test"]["Y"]
    
    reshaped_train_halflife = np.array(benchmark_train_halflives).reshape(-1, 1)
    scaler = MinMaxScaler().fit(reshaped_train_halflife)
    train_halflife_scaled = scaler.transform(reshaped_train_halflife)
    train_halflives_scaled = np.array([val[0] for val in train_halflife_scaled])

    reshaped_test_halflife = np.array(benchmark_test_halflives).reshape(-1, 1)
    scaler = MinMaxScaler().fit(reshaped_test_halflife)
    test_halflife_scaled = scaler.transform(reshaped_test_halflife)
    test_halflives_scaled = np.array([val[0] for val in test_halflife_scaled])
    
    halflives["morgan"][benchmark]["train"] = np.array(train_halflives_scaled)
    halflives["morgan"][benchmark]["test"] = np.array(test_halflives_scaled)
    
    print(halflives["morgan"].keys())
    
    print(benchmark_train_smiles.shape, benchmark_train_halflives.shape, benchmark_test_smiles.shape, benchmark_test_halflives.shape)

obach
dict_keys(['obach'])
(467,) (467,) (133,) (133,)
microsome
dict_keys(['obach', 'microsome'])
(772,) (772,) (220,) (220,)
hepatocyte
dict_keys(['obach', 'microsome', 'hepatocyte'])
(849,) (849,) (243,) (243,)


In [5]:
mol_features["morgan"] = {}
for benchmark in tdc_benchmarks:
    print(benchmark)
    mol_features["morgan"][benchmark] = {}
    train_morgan_fps = np.array(fp_from_smiles(smiles[benchmark]["train"]))
    test_morgan_fps = np.array(fp_from_smiles(smiles[benchmark]["test"]))
    mol_features["morgan"][benchmark]["train"] = train_morgan_fps
    mol_features["morgan"][benchmark]["test"] = test_morgan_fps
    print(train_morgan_fps.shape, test_morgan_fps.shape)

obach
(467, 124) (133, 124)
microsome
(772, 124) (220, 124)
hepatocyte
(849, 124) (243, 124)


In [6]:
mol_features["jazzy"] = {}
halflives["jazzy"] = {}
for benchmark in tdc_benchmarks:
    print(benchmark)
    train_jazzy_df = pd.read_csv(f"project_resources/jazzy_splits/{benchmark}_train.csv")
    test_jazzy_df = pd.read_csv(f"project_resources/jazzy_splits/{benchmark}_test.csv")
    train_fts, train_jazzy_thalfs, contains_nan = parse_jazzy_df(train_jazzy_df, no_idx_smi=True)
    test_fts, test_jazzy_thalfs, contains_nan = parse_jazzy_df(test_jazzy_df, no_idx_smi=True)
    
    mol_features["jazzy"][benchmark] = {}
    mol_features["jazzy"][benchmark]["train"] = train_fts
    mol_features["jazzy"][benchmark]["test"] = test_fts
    halflives["jazzy"][benchmark] = {}
    halflives["jazzy"][benchmark]["train"] = train_jazzy_thalfs
    halflives["jazzy"][benchmark]["test"] = test_jazzy_thalfs
    
    print(np.array(train_fts).shape, np.array(train_jazzy_thalfs).shape, np.array(test_fts).shape, np.array(test_jazzy_thalfs).shape)

obach
     525, [0.0033169983665033, 11.3305, 0.0, 3.3351, -4.054, -69.5826, -60.2942]
     130, [0.0078064793190067, 10.9705, 1.8136, 5.8249, -16.4281, -118.9807, -121.6516]
(525, 6) (525,) (130, 6) (130,)
microsome
     882, [0.0652380952380952, 10.5072, 1.4478, 4.6964, -13.5025, -92.8889, -106.3914]
     220, [0.0884353741496598, 3.5084, 3.5128, 4.5042, -12.6756, -80.3865, -84.2257]
(882, 6) (882,) (220, 6) (220,)
hepatocyte
     970, [0.0, 9.8552, 1.4451, 4.4407, -15.1209, -91.0733, -102.926]
     243, [0.0825850340136054, 10.2098, 1.292, 5.2199, -18.4498, -95.9609, -98.199]
(970, 6) (970,) (243, 6) (243,)


In [7]:
sampler = samplers['TPESampler']
pruner = pruners["BasePruner"]
n_trials = 200
with ThreadPoolExecutor() as executor:
    futures = []
    for _type in feature_types:
        for benchmark in tdc_benchmarks:
            X_train = mol_features[_type][benchmark]["train"]
            y_train = halflives[_type][benchmark]["train"]
            X_test = mol_features[_type][benchmark]["test"]
            y_test = halflives[_type][benchmark]["test"]

            for model_identifier in model_identifiers:
                print(_type, benchmark, model_identifier)
                lock_obj = optuna.storages.JournalFileOpenLock(
                    f"./project_resources/optuna/{benchmark}/{_type}/{model_identifier}_journal.log"
                )

                storage = JournalStorage(
                                JournalFileStorage(f"./project_resources/optuna/{benchmark}/{_type}/{model_identifier}_journal.log", lock_obj=lock_obj)
                            )
                study = optuna.create_study(study_name=model_identifier, directions=['minimize'], pruner=pruner,
                                            storage=storage, load_if_exists=True)
                
                trial_log_csv_path = f"./project_resources/optuna/{benchmark}/{_type}/{model_identifier}.csv"
                tuner = HyperparamTuner(trial_log_csv_path, model_identifier, X_train, y_train, X_test, y_test)
                
                futures.append(executor.submit(study.optimize, tuner.objective, n_trials=n_trials))
                joblib.dump(study, f"./project_resources/optuna/{benchmark}/{_type}/{model_identifier}.pkl")
    for future in futures:
        future.result()

[I 2023-12-02 15:12:17,346] Using an existing study with name 'linear' instead of creating a new one.
[I 2023-12-02 15:12:17,362] Using an existing study with name 'KRR' instead of creating a new one.
[I 2023-12-02 15:12:17,380] Using an existing study with name 'GB' instead of creating a new one.
[I 2023-12-02 15:12:17,400] Using an existing study with name 'RF' instead of creating a new one.
[I 2023-12-02 15:12:17,425] Using an existing study with name 'ANN' instead of creating a new one.
[I 2023-12-02 15:12:17,440] Trial 2 finished with value: 0.06194177088509379 and parameters: {'alpha': 0.050063927670237256, 'l1_ratio': 0.017851664721826643}. Best is trial 2 with value: 0.06194177088509379.


morgan obach linear
morgan obach KRR
morgan obach GB
morgan obach RF
morgan obach ANN
Successfully updated linear.csv with results of trial 2


[I 2023-12-02 15:12:17,481] Using an existing study with name 'linear' instead of creating a new one.
[I 2023-12-02 15:12:17,510] Using an existing study with name 'KRR' instead of creating a new one.


morgan microsome linear
morgan microsome KRR


[I 2023-12-02 15:12:17,543] Using an existing study with name 'GB' instead of creating a new one.
[I 2023-12-02 15:12:17,579] Using an existing study with name 'RF' instead of creating a new one.
[I 2023-12-02 15:12:17,629] Using an existing study with name 'ANN' instead of creating a new one.
[I 2023-12-02 15:12:17,647] Trial 2 finished with value: 0.08153615538053395 and parameters: {'alpha': 0.44691318753335496, 'gamma': 8.781933727089157e-15, 'kernel': 'laplacian'}. Best is trial 0 with value: 0.08091152083395386.


...

Successfully updated ANN.csv with results of trial 201


[I 2023-12-02 17:34:11,862] Trial 200 finished with value: 0.3818726551412707 and parameters: {'learning_rate_init': 0.07180845410627103, 'hidden_layer_sizes': [10, 10]}. Best is trial 11 with value: 0.3741264197111395.


Successfully updated ANN.csv with results of trial 200


[I 2023-12-02 17:34:17,327] Trial 201 finished with value: 0.37680861828505646 and parameters: {'learning_rate_init': 0.008239485096309018, 'hidden_layer_sizes': [20, 20]}. Best is trial 11 with value: 0.3741264197111395.


Successfully updated ANN.csv with results of trial 201
